In [1]:
# start by importing the modules we will need
import requests
import json  #we will use these later in retrieving geodata and plotting university locations using openstreetmaps
import folium
import numpy as np
import cufflinks as cf
import pandas as pd #These allow us to open our data in a pandas data frame
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
print("Ready to go to college?".title())

Ready To Go To College?


In [2]:
CENTER_US = (39.8333333,-98.585522)
#london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)

In [3]:
import sys
from sys import exit

help(exit)

Help on built-in function exit in module sys:

exit(...)
    exit([status])
    
    Exit the interpreter by raising SystemExit(status).
    If the status is omitted or None, it defaults to zero (i.e., success).
    If the status is an integer, it will be used as the system exit status.
    If it is another kind of object, it will be printed and the system
    exit status will be one (i.e., failure).



In [4]:
#accepted_states = ['Alabama','Alaska','Arizona','Arkansas' ,'California' ,'Colorado','Connecticut', 'Delaware','Florida','Georgia']
state = input('Enter State: ex. NewYork ')
try:
    state = pd.read_csv(state)
except FileNotFoundError:
    print('Please re-run and follow the correct format. Enter the state in title format. If the state has two words, ignore the space.')
    print("For example: Connecticut, NewYork, NewJersey, etc.")
    try:
        state = input('Enter State: ex. NewYork ')
        state = pd.read_csv(state)
    except FileNotFoundError:
        exit(0)
    

    #while True:
        #state = input('Enter State: ex. NewYork ')
        #if state is not in accepted_states:
            #break
        #else:
            #state = pd.read_csv(state)
#Add column based on size
state['size'] = np.nan
state['size'][state['Student population']>15000] = 'large'
state['size'][state['Student population']<=15000] = 'medium'
state['size'][state['Student population'] <=5000] = 'small'
state

display(state)
#filter by type of institution
college_types = list(state['Type'].value_counts().index)
i = 1
for types in college_types:
    print(i,types)
    i = i + 1
choice = input ('What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. ')
try:
    if choice == '':
        state2 = state
    else:
        choice = int(choice)#convert choice into an integer that corresponds to the menu options
        choice_value = college_types[choice-1]
        state2 = state[state['Type']==choice_value]
    display(state2)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state2 = state

#filter by surrounding area
surrounding_area = list(state2['Campus setting'].value_counts().index)
i = 1
for types in surrounding_area:
    print(i,types)
    i = i + 1
area = input ('What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.')
try:
    if area == '':
        state3 = state2
    else:
        area = int(area)
        setting_preference = surrounding_area[area-1]
        state3 = state2[state2['Campus setting']==setting_preference]
    display(state3)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state3=state2

#filter by size
college_sizes = list(state3['size'].value_counts().index)
i = 1
for types in college_sizes:
    print(i,types)
    i = i + 1
size_preference = input("How big do you want your university to be? Above are your options. Press enter if you have no preference.")
try:
    if size_preference == '':
        state4 = state3
    else:
        size_preference= int(size_preference) 
        size = college_sizes[size_preference -1]
        state4=state3[state3['size']==size]
    display(state4)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state4=state3

#filter by on campus housing
campus_housing = input('Do you want campus housing? Yes or no or type anything you do not have a preference. ')
campus_housing = campus_housing.capitalize()

if campus_housing == 'Yes':
    state5 = state4[state4['Campus housing'] == 'Yes']
    display(state5)
elif campus_housing == 'No':
    state5 = state4[state4['Campus housing'] == 'No']
    display(state5)
else: 
    state5 = state4
    display(state5)    


Enter State: ex. NewYork Delaware


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
0,Delaware State University,"1200 N. Dupont Highway, Dover, Delaware 19901",www.desu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,City: Small,Yes,4328.0,3993.0,43%,NaN,Fall 2010,"$12,221",-,130934.0,142800.0,small
1,Delaware Technical Community College-Terry,"100 Campus Drive, Dover, Delaware 19901",https://www.dtcc.edu/our-campuses/dover,"4-year, primarily associate's, Public",Less than one year certificate|One but less th...,City: Small,No,14479.0,14479.0,13%,9%,Fall 2010,"$7,003",-,130907.0,1172700.0,medium
2,Goldey-Beacom College,"4701 Limestone Rd, Wilmington, Delaware 19808",www.gbc.edu,"4-year, Private not-for-profit",Less than one year certificate|One but less th...,Suburb: Large,Yes,2063.0,698.0,68%,10%,Fall 2010,"$15,718",-,130989.0,142900.0,small
3,Strayer University-Delaware,"240 Continental Dr Ste 108, Newark, Delaware 1...",www.strayer.edu/delaware/christiana,"4-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,301.0,246.0,NaN,NaN,Fall 2010,"$25,961",-,450298.0,145948.0,small
4,University of Delaware,"104 Hullihen Hall, Newark, Delaware 19716",www.udel.edu/,"4-year, Public",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,Yes,23009.0,19215.0,81%,NaN,Fall 2010,"$15,894",-,130943.0,143100.0,large
5,Wesley College,"120 N State St, Dover, Delaware 19901-3875",www.wesley.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Postbacca...,City: Small,Yes,1473.0,1384.0,22%,NaN,Fall 2010,"$23,619",-,131098.0,143300.0,small
6,Wilmington University,"320 Dupont Hwy, New Castle, Delaware 19720",www.wilmu.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,No,15316.0,8862.0,33%,35%,Fall 2010,"$14,471",-,131113.0,794800.0,large
7,NaN,* Cohort Year represents the year for which st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,** Net Price is Average Net Price for 2015-201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"NA (Not applicable) - Graduation rate, transfe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1 4-year, Private not-for-profit
2 4-year, Public
3 4-year, Private for-profit
4 4-year, primarily associate's, Public
What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. 


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
0,Delaware State University,"1200 N. Dupont Highway, Dover, Delaware 19901",www.desu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,City: Small,Yes,4328.0,3993.0,43%,NaN,Fall 2010,"$12,221",-,130934.0,142800.0,small
1,Delaware Technical Community College-Terry,"100 Campus Drive, Dover, Delaware 19901",https://www.dtcc.edu/our-campuses/dover,"4-year, primarily associate's, Public",Less than one year certificate|One but less th...,City: Small,No,14479.0,14479.0,13%,9%,Fall 2010,"$7,003",-,130907.0,1172700.0,medium
2,Goldey-Beacom College,"4701 Limestone Rd, Wilmington, Delaware 19808",www.gbc.edu,"4-year, Private not-for-profit",Less than one year certificate|One but less th...,Suburb: Large,Yes,2063.0,698.0,68%,10%,Fall 2010,"$15,718",-,130989.0,142900.0,small
3,Strayer University-Delaware,"240 Continental Dr Ste 108, Newark, Delaware 1...",www.strayer.edu/delaware/christiana,"4-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,301.0,246.0,NaN,NaN,Fall 2010,"$25,961",-,450298.0,145948.0,small
4,University of Delaware,"104 Hullihen Hall, Newark, Delaware 19716",www.udel.edu/,"4-year, Public",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,Yes,23009.0,19215.0,81%,NaN,Fall 2010,"$15,894",-,130943.0,143100.0,large
5,Wesley College,"120 N State St, Dover, Delaware 19901-3875",www.wesley.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Postbacca...,City: Small,Yes,1473.0,1384.0,22%,NaN,Fall 2010,"$23,619",-,131098.0,143300.0,small
6,Wilmington University,"320 Dupont Hwy, New Castle, Delaware 19720",www.wilmu.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,No,15316.0,8862.0,33%,35%,Fall 2010,"$14,471",-,131113.0,794800.0,large
7,NaN,* Cohort Year represents the year for which st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,** Net Price is Average Net Price for 2015-201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"NA (Not applicable) - Graduation rate, transfe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1 Suburb: Large
2 City: Small
What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
0,Delaware State University,"1200 N. Dupont Highway, Dover, Delaware 19901",www.desu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,City: Small,Yes,4328.0,3993.0,43%,NaN,Fall 2010,"$12,221",-,130934.0,142800.0,small
1,Delaware Technical Community College-Terry,"100 Campus Drive, Dover, Delaware 19901",https://www.dtcc.edu/our-campuses/dover,"4-year, primarily associate's, Public",Less than one year certificate|One but less th...,City: Small,No,14479.0,14479.0,13%,9%,Fall 2010,"$7,003",-,130907.0,1172700.0,medium
2,Goldey-Beacom College,"4701 Limestone Rd, Wilmington, Delaware 19808",www.gbc.edu,"4-year, Private not-for-profit",Less than one year certificate|One but less th...,Suburb: Large,Yes,2063.0,698.0,68%,10%,Fall 2010,"$15,718",-,130989.0,142900.0,small
3,Strayer University-Delaware,"240 Continental Dr Ste 108, Newark, Delaware 1...",www.strayer.edu/delaware/christiana,"4-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,301.0,246.0,NaN,NaN,Fall 2010,"$25,961",-,450298.0,145948.0,small
4,University of Delaware,"104 Hullihen Hall, Newark, Delaware 19716",www.udel.edu/,"4-year, Public",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,Yes,23009.0,19215.0,81%,NaN,Fall 2010,"$15,894",-,130943.0,143100.0,large
5,Wesley College,"120 N State St, Dover, Delaware 19901-3875",www.wesley.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Postbacca...,City: Small,Yes,1473.0,1384.0,22%,NaN,Fall 2010,"$23,619",-,131098.0,143300.0,small
6,Wilmington University,"320 Dupont Hwy, New Castle, Delaware 19720",www.wilmu.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,Suburb: Large,No,15316.0,8862.0,33%,35%,Fall 2010,"$14,471",-,131113.0,794800.0,large
7,NaN,* Cohort Year represents the year for which st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,** Net Price is Average Net Price for 2015-201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"NA (Not applicable) - Graduation rate, transfe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1 small
2 large
3 medium
How big do you want your university to be? Above are your options. Press enter if you have no preference.3


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
1,Delaware Technical Community College-Terry,"100 Campus Drive, Dover, Delaware 19901",https://www.dtcc.edu/our-campuses/dover,"4-year, primarily associate's, Public",Less than one year certificate|One but less th...,City: Small,No,14479.0,14479.0,13%,9%,Fall 2010,"$7,003",-,130907.0,1172700.0,medium


Do you want campus housing? Yes or no or type anything you do not have a preference. no


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
1,Delaware Technical Community College-Terry,"100 Campus Drive, Dover, Delaware 19901",https://www.dtcc.edu/our-campuses/dover,"4-year, primarily associate's, Public",Less than one year certificate|One but less th...,City: Small,No,14479.0,14479.0,13%,9%,Fall 2010,"$7,003",-,130907.0,1172700.0,medium


In [6]:
school_data= pd.DataFrame({'Name':state4['Name'],
              'Location':state4['Address'],'Website': state4['Website']})
display(school_data)
#list(school_data)

#created a new data frame with the information we want. 



def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in school_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


,Location,Name,Website
1,"100 Campus Drive, Dover, Delaware 19901",Delaware Technical Community College-Terry,https://www.dtcc.edu/our-campuses/dover


In [9]:
i = 0

all_data= pd.DataFrame({'Name':state['Name'],
              'Location':state['Address'],'Website': state['Website']})
def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in all_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


Try running me again.
